In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd

In [2]:
import sys
sys.path.append("/Users/exilis/work/backlight/src")

In [3]:
import backlight

# Generate example dummy data

In [4]:
np.random.seed(0)

# market data
if not os.path.exists("example_market.csv"):
    idx = pd.date_range("2018-04-01 00:00:00", "2018-06-30 23:59:59", freq="10S")
    ask = np.cumsum(np.random.rand(len(idx)) - 0.5) + 100.0
    bid = ask - 0.02
    pd.DataFrame(
        index=idx,
        data=np.array([ask, bid]).T,
        columns=["ask", "bid"]
    ).abs().to_csv("example_market.csv")

# signal data
if not os.path.exists("example_signal.csv"):
    sig = np.random.rand(3, len(idx)).T
    n = np.sum(sig, axis=1)
    pd.DataFrame(
        index=idx,
        data=sig / n.reshape(-1, 1),
        columns=["down", "neutral", "up"]
    ).to_csv("example_signal.csv")

# Configuration

In [5]:
model_id = ""
start_dt = "2018-06-03 00:00:00+0000"
end_dt = "2018-06-10 00:00:00+0000"
sig_url = "file:///{}/example_signal.csv".format(os.getcwd())
mkt_url = "file:///{}/example_market.csv".format(os.getcwd())

# loading the signal

In [6]:
from backlight import signal

SYMBOL = "USDJPY"
sig = signal.load_signal(SYMBOL, sig_url, start_dt, end_dt)

In [7]:
sig.tail()

,down,neutral,up,pred
2018-06-09 23:59:20,0.148715,0.551230,0.300055,0.0
2018-06-09 23:59:30,0.157524,0.698833,0.143643,0.0
2018-06-09 23:59:40,0.424951,0.315313,0.259736,-1.0
2018-06-09 23:59:50,0.353674,0.515737,0.130589,0.0
2018-06-10 00:00:00,0.408115,0.465368,0.126517,0.0


In [8]:
sig.symbol

'USDJPY'

# loading the marketdata

In [9]:
from backlight import datasource

mkt = datasource.load_marketdata(
    sig.symbol,
    sig.start_dt,
    sig.end_dt,
    mkt_url,
)

In [10]:
mkt.tail()

,ask,bid
2018-06-09 23:59:20,39.916218,39.896218
2018-06-09 23:59:30,39.750804,39.730804
2018-06-09 23:59:40,39.932591,39.912591
2018-06-09 23:59:50,39.433785,39.413785
2018-06-10 00:00:00,39.471854,39.451854


# generating labels

## Fixed Neutral

In [11]:
from backlight import labelizer


lbl_fix = labelizer.generate_labels(
    mkt,
    labelizer.FixedNeutralLabelizer(
        lookahead="3Min",
        neutral_atol=0.075,
        neutral_rtol=0.00,
    ))

In [12]:
lbl_fix.head()

,label_diff,label
2018-06-03 00:00:00,-0.854094,-1.0
2018-06-03 00:00:10,-0.446213,-1.0
2018-06-03 00:00:20,0.156435,1.0
2018-06-03 00:00:30,-0.402531,-1.0
2018-06-03 00:00:40,-0.609441,-1.0


In [13]:
lbl_fix.label_type

'ternary'

In [15]:
# (-1.0, 0.0, 1.0) : (Down, Neutral, Up)
lbl_fix.groupby("label").label.count() / len(lbl_fix)

label
-1.0    0.479952
 0.0    0.047751
 1.0    0.471999
Name: label, dtype: float64

## Dynamic Neutral

In [16]:
lbl_dyn = labelizer.generate_labels(
    mkt,
    labelizer.DynamicNeutralLabelizer(
        lookahead="3Min",
        neutral_ratio=0.38,
        neutral_window="1H",
        neutral_hard_limit=0.0,
    ))

In [17]:
lbl_dyn.head()

,label_diff,label,neutral_range
2018-06-03 00:00:00,-0.854094,-1.0,0.854094
2018-06-03 00:00:10,-0.446213,0.0,0.601208
2018-06-03 00:00:20,0.156435,0.0,0.376667
2018-06-03 00:00:30,-0.402531,0.0,0.408647
2018-06-03 00:00:40,-0.609441,-1.0,0.425246


In [18]:
lbl_dyn.label_type

'ternary'

In [19]:
# (-1.0, 0.0, 1.0) : (Down, Neutral, Up)
lbl_dyn.groupby("label").label.count() / len(lbl_dyn)

label
-1.0    0.310511
 0.0    0.382798
 1.0    0.306394
Name: label, dtype: float64

# simulate trading strategy

In [20]:
from backlight import strategies
trades = strategies.simple_buy_sell(mkt, sig, target_column_name="???")

In [21]:
trades.head()

,amount,ask,bid
2018-06-03 00:00:00,0.0,80.267283,80.247283
2018-06-03 00:00:10,0.0,80.020294,80.000294
2018-06-03 00:00:20,1.0,79.711448,79.691448
2018-06-03 00:00:30,0.0,79.832766,79.812766
2018-06-03 00:00:40,1.0,79.969499,79.949499


# simulate and evaluate positions

In [22]:
from backlight import positions
positions = positions.calc_positions(trades, mkt)

In [23]:
positions.head()

,ask,bid,amount,price
2018-06-03 00:00:00,80.267283,80.247283,0.0,80.257283
2018-06-03 00:00:10,80.020294,80.000294,0.0,80.010294
2018-06-03 00:00:20,79.711448,79.691448,1.0,79.701448
2018-06-03 00:00:30,79.832766,79.812766,1.0,79.822766
2018-06-03 00:00:40,79.969499,79.949499,2.0,79.959499


# Calculate Metrics - based on the raw signals

In [24]:
from backlight import metrics
m = metrics.calc_metrics(sig, lbl_fix)
m

,metrics
cnt_uu,9489.000000
cnt_un,962.000000
cnt_ud,9676.000000
cnt_nu,9489.000000
cnt_nn,962.000000
cnt_nd,9676.000000
cnt_du,9489.000000
cnt_dn,962.000000
cnt_dd,9676.000000
cnt_total,60481.000000


In [25]:
from backlight import metrics
m = metrics.calc_metrics(sig, lbl_dyn)
m

,metrics
cnt_uu,6139.000000
cnt_un,7655.000000
cnt_ud,6277.000000
cnt_nu,6139.000000
cnt_nn,7655.000000
cnt_nd,6277.000000
cnt_du,6139.000000
cnt_dn,7655.000000
cnt_dd,6277.000000
cnt_total,60481.000000
